In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
root="dataset/fashionMNIST",
train=True,
download=False,
transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="dataset/fashionMNIST",
    train=False,
    download=False,
    transform=ToTensor()
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for x, y in test_dataloader:
    print(f"shape of x [N, C, H, W]: {x.shape}")
    print(f"shape of y: {y.shape} {y.dtype}")
    break

shape of x [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]) torch.int64


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    # else "mps"
    # if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device=device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optmizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloadr, model, loss_fn, optmizer):
    size = len(dataloadr.dataset)
    for batch, (x, y) in enumerate(dataloadr):
        x, y = x.to(device), y.to(device)
        
        pred = model(x)
        loss = loss_fn(pred, y)
        
        optmizer.zero_grad()
        loss.backward()
        optmizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f} [{current:>5f}/{size:>5f}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train(train_dataloader, model, loss_fn, optmizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 2.294853 [64.000000/60000.000000]


loss: 2.285164 [6464.000000/60000.000000]
loss: 2.266148 [12864.000000/60000.000000]
loss: 2.257408 [19264.000000/60000.000000]
loss: 2.245326 [25664.000000/60000.000000]
loss: 2.221223 [32064.000000/60000.000000]
loss: 2.220344 [38464.000000/60000.000000]
loss: 2.198640 [44864.000000/60000.000000]
loss: 2.183513 [51264.000000/60000.000000]
loss: 2.156610 [57664.000000/60000.000000]
Test Error: 
 Accuracy: 48.8%, Avg loss: 2.143994 

Epoch 2
----------------------------
loss: 2.158234 [64.000000/60000.000000]
loss: 2.144165 [6464.000000/60000.000000]
loss: 2.086643 [12864.000000/60000.000000]
loss: 2.094342 [19264.000000/60000.000000]
loss: 2.055792 [25664.000000/60000.000000]
loss: 1.996209 [32064.000000/60000.000000]
loss: 2.018709 [38464.000000/60000.000000]
loss: 1.951524 [44864.000000/60000.000000]
loss: 1.943341 [51264.000000/60000.000000]
loss: 1.875493 [57664.000000/60000.000000]
Test Error: 
 Accuracy: 54.3%, Avg loss: 1.866394 

Epoch 3
----------------------------
loss: 1.90

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[torch.argmax(pred[0]).cpu().item()], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
